# Imports

In [ ]:
import configparser
import requests
import datetime
import numpy as np
from gtts import gTTS
from googletrans import Translator

# Constants

In [ ]:
API_KEY = '96f16829e533921c155017d84728007b'
CITIES = [('gourcy', 13.2167, -2.35), 
          ('bassi', 13.3252, -2.2453), 
          ('bougounam', 13.362, -2.3892),
          ('london', 51.51, -0.13),
          ('amsterdam', 52.3792, 4.8994)]

# Functions: Quantative --> Qualitative

In [ ]:
def temp_qual(temperature):
    '''
    Converts temperature in Celsius to a qualitative scale.
    '''
    if temperature < 16:
        return 'cold'
    elif temperature < 20:
        return 'neutral'
    elif temperature < 25:
        return 'warm'
    elif temperature < 40:
        return 'hot'
    elif temperature >= 40:
        return 'very hot'
    else:
        return None


def wind_qual(windspeed):
    '''
    Converts windspeed in m/s to a qualitative (beaufort) scale.    
    '''
    if windspeed < 0.3:
        return 'no wind'
    elif windspeed < 1.6:
        return 'light air'
    elif windspeed < 3.4:
        return 'light breeze'
    elif windspeed < 5.5:
        return 'gentle breeze'
    elif windspeed < 8.0:
        return 'moderate breeze'
    elif windspeed < 10.8:
        return 'fresh breeze'
    elif windspeed < 13.9:
        return 'strong breeze'
    elif windspeed < 17.2:
        return 'high wind'
    elif windspeed < 20.8:
        return 'gale'
    elif windspeed < 24.5:
        return 'strong gale'
    elif windspeed < 28.5:
        return 'storm'
    elif windspeed < 32.7:
        return 'violent storm'
    elif windspeed >= 32.7:
        return 'hurricane'
    else:
        return None


def rain_qual(rain):
    '''
    Converts rain in mm/day to a qualitative scale.    
    '''    
    if rain == 0:
        return 'no'
    elif rain < 2:
        return 'light'
    elif rain < 10:
        return 'moderate'
    elif rain < 50:
        return 'heavy'
    elif rain >= 50:
        return 'very heavy'
    else:
        return None

# Functions: Misc

In [ ]:
def converttimestamp(dt):
    '''
    Converts unix timestamp to readable date and time
    '''
    timestamp = datetime.datetime.fromtimestamp(dt)
    return timestamp.strftime('%Y-%m-%d')

# Functions: Get and filter weather data

In [ ]:
def get_weather(lat, lon):
    '''
    Retrieves and filters the weatherforecast for the next 7 days.
    '''
    url = 'https://api.openweathermap.org/data/2.5/onecall?lat={}&lon={}&units=metric&exclude=current,hourly&appid={}'.format(lat, lon, API_KEY)
    return requests.get(url).json()['daily'][1:]


def select_weatherdata(daily_forecast):
    '''
    Selects the needed weather data within daily forecast data
    Stores as dicts per day in a list
    '''
    weatherdatalist = []
    
    for forecast in daily_forecast:
        
        weatherdata = {}
        weatherdata['dtread'] = converttimestamp(forecast['dt']) if 'dt' in forecast else None
        weatherdata['windspeed'] = forecast['wind_speed'] if 'wind_speed' in forecast else 0
        weatherdata['feeltemp'] = forecast['feels_like']['day'] if 'feels_like' in forecast else None
        weatherdata['rain'] = forecast['rain'] if 'rain' in forecast else 0
        
        weatherdatalist.append(weatherdata)
    return weatherdatalist

# Functions: Process weather data

In [ ]:
# calculate weatherdata for 1, 3 and 7 days in a dict

def calc_weatherdata(weatherlist):
    weathercalcdict = {}
    weathercalcdict['24 hours'] = {}
    weathercalcdict['3 days'] = {}
    weathercalcdict['7 days'] = {}


    # Calculate the 3 day prediction values
    day3_temp = round((weatherlist[0]['feeltemp'] + weatherlist[1]['feeltemp'] + 
                       weatherlist[2]['feeltemp']) / 3, 2)
    day3_wind = round((weatherlist[0]['windspeed'] + weatherlist[1]['windspeed'] +
                       weatherlist[2]['windspeed'])/3, 2)
    day3_rain = round( weatherlist[0]['rain'] + weatherlist[1]['rain'] + 
                       weatherlist[2]['rain'], 2)

    # Calculate the 7 day prediction values
    day7_temp = round((weatherlist[0]['feeltemp'] + weatherlist[1]['feeltemp'] + 
                       weatherlist[2]['feeltemp'] + weatherlist[3]['feeltemp'] + 
                       weatherlist[4]['feeltemp'] + weatherlist[5]['feeltemp'] + 
                       weatherlist[6]['feeltemp']) / 7, 2)
    day7_wind = round((weatherlist[0]['windspeed'] + weatherlist[1]['windspeed'] + 
                       weatherlist[2]['windspeed'] + weatherlist[3]['windspeed'] + 
                       weatherlist[4]['windspeed'] + weatherlist[5]['windspeed'] + 
                       weatherlist[6]['windspeed']) / 7, 2)
    day7_rain = round( weatherlist[0]['rain'] + weatherlist[1]['rain'] + 
                       weatherlist[2]['rain'] + weatherlist[3]['rain'] + 
                       weatherlist[4]['rain'] + weatherlist[5]['rain'] + 
                       weatherlist[6]['rain'], 2)
    
    # Add the found values to the dictionary (+ 24 hour values)
    weathercalcdict['24 hours']['windspeed'] = weatherlist[0]['windspeed']
    weathercalcdict['24 hours']['feeltemp'] = weatherlist[0]['feeltemp']
    weathercalcdict['24 hours']['rain'] = weatherlist[0]['rain']

    weathercalcdict['3 days']['windspeed'] = day3_wind
    weathercalcdict['3 days']['feeltemp'] = day3_temp
    weathercalcdict['3 days']['rain'] = day3_rain

    weathercalcdict['7 days']['windspeed'] = day7_wind
    weathercalcdict['7 days']['feeltemp'] = day7_temp
    weathercalcdict['7 days']['rain'] = day7_rain
    return weathercalcdict

def conv_weatherdata(weatherdict):
    '''
    Converts quantitative values to qualitative categories
    '''    
    
    for timeframe in weatherdict:
        weatherdict[timeframe] = {'feeltemp': temp_qual(weatherdict[timeframe]['feeltemp']),
                                  'windspeed': wind_qual(weatherdict[timeframe]['windspeed']),
                                  'rain': rain_qual(weatherdict[timeframe]['rain'])}   
    return weatherdict

# Functions: Formulate and save forecast

In [ ]:
def formulate_forecast(weatherdict, city):
    for timeframe in weatherdict:
        temp = weatherdict[timeframe]['feeltemp']
        wind = weatherdict[timeframe]['windspeed']
        rain = weatherdict[timeframe]['rain']        
        
        weatherdict[timeframe] = 'The weather for %s for the next %s. The temperature will be %s, there will be a %s and %s rain' % (city, timeframe, temp, wind, rain)
        
    return weatherdict


def forecast_to_speech(dict_forecasts):
    for city in dict_forecasts:
        for timeframe in dict_forecasts[city]:
            
            # English
            txt_en = dict_forecasts[city][timeframe]
            tts_en = gTTS(txt_en)
            tts_en.save(f'{city}_{timeframe}_EN.mp3')
            
            # French
            translator = Translator()
            txt_fr = translator.translate(txt_en, dest='fr')
            tts_fr = gTTS(txt_fr.text, lang='fr')
            tts_fr.save(f'{city}_{timeframe}_FR.mp3')

# Functions: Main function

In [ ]:
def main(save_audio = False):
    convert_audio = {}
    
    for city, latitude, longitude in CITIES:
        weather = get_weather(latitude, longitude)
        filtered = select_weatherdata(weather)
        calculated = calc_weatherdata(filtered)
        converted = conv_weatherdata(calculated)
        forecasts = formulate_forecast(converted, city)
        
        convert_audio[city] = forecasts
    
    if save_audio:
        forecast_to_speech(convert_audio)
    else:
        return convert_audio
        

In [ ]:
main(save_audio=False)